In [1]:
from datasets import load_from_disk
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
from datasets import load_from_disk
from eval.cola import evaluate, generate_answers_batch
import torch
from sklearn.metrics import accuracy_score


2025-04-26 22:09:34.663082: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-26 22:09:34.669690: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745723374.677881 1395646 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745723374.680269 1395646 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-26 22:09:34.689760: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
# Load the model and tokenizer
MODEL_ID = "mistralai/Mistral-7B-v0.1"

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    torch_dtype=torch.float16,
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
# add adapter, if not then base model
# model = base_model

# load pretrained adapter
# peft_model_gsm8k_id = "predibase/gsm8k"
# peft_model_magicoder_id = "predibase/magicoder"
# peft_model_gluecola_id = "predibase/glue_cola"
# peft_model_hellaswag_id = "predibase/hellaswag"

# adapter_name_gsm8k = "gsm8k"
# adapter_name_magicoder = "magicoder"
# adapter_name_gluecola = "glue_cola"
# adapter_name_hellaswag = "hellaswag"

# model = PeftModel.from_pretrained( # not loading from local
#     base_model,
#     peft_model_hellaswag_id,
#     adapter_name=adapter_name_hellaswag, # You can name the first adapter here
#     device_map="auto", # Apply device mapping here if needed
#     # low_cpu_mem_usage=True,
#     # offload_folder='offload/'
# )

# load merged adapter
model = PeftModel.from_pretrained(
    base_model,
    "./weights/element_add/gsm8k_magicoder_hellaswag/gsm8k_magicoder_hellaswag",
    device_map='auto'
    )
model = model.half()        # convert to half precision (if your GPU supports it)
model = torch.compile(model) 
model.eval()


This can be used to load a bitsandbytes version that is different from the PyTorch CUDA version.
If this was unintended set the BNB_CUDA_VERSION variable to an empty string: export BNB_CUDA_VERSION=
If you use the manual override make sure the right libcudart.so is in your LD_LIBRARY_PATH
For example by adding the following to your .bashrc: export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:<path_to_cuda_dir/lib64



OptimizedModule(
  (_orig_mod): PeftModelForCausalLM(
    (base_model): LoraModel(
      (model): MistralForCausalLM(
        (model): MistralModel(
          (embed_tokens): Embedding(32000, 4096)
          (layers): ModuleList(
            (0-31): 32 x MistralDecoderLayer(
              (self_attn): MistralAttention(
                (q_proj): lora.Linear(
                  (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.05, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=4096, out_features=8, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=8, out_features=4096, bias=False)
                  )
                  (lora_embedding_A): ParameterDict()
                  (lora_embedding_B): ParameterDict()
                  (lora_magn

In [4]:
dataset = load_from_disk("data/blimp_adjunct_anaphor_refined")
dataset['sentence'][0]

"Tara hasn't cured themselves."

In [6]:
prompt_template = (
    "Is the following sentence grammatically correct?\n"
    "Sentence: \"{sentence}\"\n"
    "Answer:"
)

all_preds = []
all_labels = []

for example in tqdm(dataset):
    sentence = example["sentence"]
    label = example["label"]
    
    prompt = prompt_template.format(sentence=sentence)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=3)
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True).lower()
    print(response)
    print(label)
    # Crude but effective classification
    if "yes" in response:
        pred = 1
    elif "no" in response:
        pred = 0
    else:
        pred = 0  # default fallback

    all_preds.append(pred)
    all_labels.append(label)
    if len(all_preds) == 10:
        break
    
accuracy = accuracy_score(all_labels, all_preds)
print(f"Accuracy (CoLA): {accuracy:.4f}")

  0%|          | 0/6000 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  0%|          | 2/6000 [00:00<06:50, 14.60it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  0%|          | 4/6000 [00:00<06:29, 15.39it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


is the following sentence grammatically correct?
sentence: "tara hasn't cured themselves."
answer:
0
is the following sentence grammatically correct?
sentence: "this government alarms themselves."
answer:
0
is the following sentence grammatically correct?
sentence: "who had every actress hidden before scaring jessica?"
answer:
1
is the following sentence grammatically correct?
sentence: "barbara isn't escaping from himself."
answer:
0


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  0%|          | 6/6000 [00:00<06:22, 15.65it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  0%|          | 8/6000 [00:00<06:18, 15.82it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


is the following sentence grammatically correct?
sentence: "edward had distracted themselves."
answer:
0
is the following sentence grammatically correct?
sentence: "the lutherans found itself."
answer:
0
is the following sentence grammatically correct?
sentence: "patricia scared himself."
answer:
0
is the following sentence grammatically correct?
sentence: "kevin doesn't care for himself."
answer:
1


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
  0%|          | 9/6000 [00:00<07:03, 14.14it/s]

is the following sentence grammatically correct?
sentence: "who can this cart worry after upsetting waiters?"
answer:
1
is the following sentence grammatically correct?
sentence: "what was debra breaking after cleaning some forks?"
answer:
1
Accuracy (CoLA): 0.6000


In [5]:
evaluate(
    model=model,
    tokenizer=tokenizer,
    dataset=dataset,
    batch_size=32
    )

100%|██████████| 188/188 [00:48<00:00,  3.84it/s]


{'accuracy': 0.0}